In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import sys
from datetime import datetime
import numpy as np
sys.path.append("../")
sys.path.append("../../")
torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, add_prefix_space=True, local_files_only = False)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            return_dict_in_generate=True,
                                            pad_token_id=tokenizer.eos_token_id).to(device)

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
symbols_important = ["He", "She"]
symbols_dummy = ["died.", 
                 "died yesterday.",
                 "died yesterday in.", 
                 "died yesterday in a.", 
                 "died yesterday in a very.", 
                 "died yesterday in a very sad.", 
                 "died yesterday in a very sad way.", 
                 "died yesterday in a very sad way due.", 
                 "died yesterday in a very sad way due to.", 
                 "died yesterday in a very sad way due to a.",
                 "died yesterday in a very sad way due to a heart.", 
                 "died yesterday in a very sad way due to a heart attack."]

In [3]:
print(tokenizer.encode(symbols_important))
print(symbols_important)

[679, 1375]
['He', 'She']


In [4]:
for symbols in symbols_dummy:
    tokens = tokenizer.encode(symbols)
    print(f"Symbols: {symbols}")
    print(f"Tokens: {tokens}")

Symbols: died.
Tokens: [3724, 13]
Symbols: died yesterday.
Tokens: [3724, 7415, 13]
Symbols: died yesterday in.
Tokens: [3724, 7415, 287, 13]
Symbols: died yesterday in a.
Tokens: [3724, 7415, 287, 257, 13]
Symbols: died yesterday in a very.
Tokens: [3724, 7415, 287, 257, 845, 13]
Symbols: died yesterday in a very sad.
Tokens: [3724, 7415, 287, 257, 845, 6507, 13]
Symbols: died yesterday in a very sad way.
Tokens: [3724, 7415, 287, 257, 845, 6507, 835, 13]
Symbols: died yesterday in a very sad way due.
Tokens: [3724, 7415, 287, 257, 845, 6507, 835, 2233, 13]
Symbols: died yesterday in a very sad way due to.
Tokens: [3724, 7415, 287, 257, 845, 6507, 835, 2233, 284, 13]
Symbols: died yesterday in a very sad way due to a.
Tokens: [3724, 7415, 287, 257, 845, 6507, 835, 2233, 284, 257, 13]
Symbols: died yesterday in a very sad way due to a heart.
Tokens: [3724, 7415, 287, 257, 845, 6507, 835, 2233, 284, 257, 2612, 13]
Symbols: died yesterday in a very sad way due to a heart attack.
Tokens: 

In [13]:
symbols_important = ["He", "She"]
symbols_dummy = ["died.", 
                 "died yesterday.",
                 "died yesterday in.", 
                 "died yesterday in a.", 
                 "died yesterday in a very.", 
                 "died yesterday in a very sad.", 
                 "died yesterday in a very sad way.", 
                 "died yesterday in a very sad way due.", 
                 "died yesterday in a very sad way due to.", 
                 "died yesterday in a very sad way due to a.",
                 "died yesterday in a very sad way due to a heart.", 
                 "died yesterday in a very sad way due to a heart attack."]

prompt = tokenizer.decode(tokenizer.bos_token_id) + "The doctor was very famous."
input_ids = torch.tensor(tokenizer.encode(prompt)).reshape(1, -1).to(device)

print(input_ids)

with torch.no_grad():
    output = model(input_ids)
    logits = output.logits[:, -1, :]
    probs = torch.softmax(logits, dim=-1)[0]
    
    
word_probs = {}
print(symbols_important)
symbols_important_ids = [tokenizer.encode(sym) for sym in symbols_important]
print(symbols_important_ids)
for i in symbols_important_ids :
    print(i)
    
    word_prob = probs[i]
    word_probs[tokenizer.decode(i).replace(" ","")] = word_prob.item()

    
normalized_word_probs_he_she = {}
total = sum(word_probs.values())
for word in word_probs:
    normalized_word_probs_he_she[word] = word_probs[word] / total




split_lists = [sentence.split() for sentence in symbols_dummy]

for lst in split_lists:
    print(lst)


def calculate_probabilities_with_prompt(split_lists, index, tokenizer, model, device, symbol_choosen):
    # Tomar la lista de palabras en la posición 'index' de split_lists
    sentence = split_lists[index]
    print(f"Original sentence: {sentence}")

    # Definir el prompt inicial con symbol_choosen y la primera palabra de la sentence
    prompt = tokenizer.decode(tokenizer.bos_token_id) + " The doctor was very famous." + symbol_choosen + " " + sentence[0]
    print(f"Initial prompt: {prompt}")
    
    # Iterar sobre la lista reduciendo el tamaño en cada paso
    for i in range(len(sentence)):
        # Limpiar word_probs para cada sublista
        word_probs.clear()

        # Usar solo las palabras desde el índice actual hasta el final
        current_sentence = sentence[i:]

        # Codificar los tokens y organizarlos en la forma [[token_id]]
        symbols_dummy_ids = [tokenizer.encode(sym) for sym in current_sentence]
        symbols_dummy_ids = [[token_id] for sublist in symbols_dummy_ids for token_id in sublist]

        print(f"Current sentence: {current_sentence}")
        print(f"Token IDs: {symbols_dummy_ids}")

        # Actualizar el prompt con la palabra actual de la sentence
        prompt += " " + current_sentence[0]
        print(f"Updated prompt: {prompt}")

        # Codificar el nuevo prompt en tokens
        input_ids = torch.tensor(tokenizer.encode(prompt)).reshape(1, -1).to(device)

        # Recalcular las probabilidades para los tokens actuales
        with torch.no_grad():
            output = model(input_ids)
            logits = output.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)[0]

        for token_ids in symbols_dummy_ids:
            for token in token_ids:
                word_prob = probs[token]  # Asumiendo que 'probs' ya fue calculado en el contexto del modelo
                word_probs[tokenizer.decode(token).replace(" ","")] = word_prob.item()

        # Normalizar las probabilidades
        normalized_word_probs_dummy = {}
        total = sum(word_probs.values())
        for word in word_probs:
            normalized_word_probs_dummy[word] = word_probs[word] / total

        print(f"Normalized probabilities for '{' '.join(current_sentence)}': {normalized_word_probs_dummy}")
    
    return normalized_word_probs_dummy

# Ejemplo de uso:
symbol_choosen = np.random.choice(list(normalized_word_probs_he_she.keys()), 
                                  p=list(normalized_word_probs_he_she.values()))
normalized_word_probs_dummy = calculate_probabilities_with_prompt(split_lists, 1, tokenizer, model, device, symbol_choosen)


tensor([[50256,   383,  6253,   373,   845,  5863,    13]])
['He', 'She']
[[679], [1375]]
[679]
[1375]
['died.']
['died', 'yesterday.']
['died', 'yesterday', 'in.']
['died', 'yesterday', 'in', 'a.']
['died', 'yesterday', 'in', 'a', 'very.']
['died', 'yesterday', 'in', 'a', 'very', 'sad.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a', 'heart.']
['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a', 'heart', 'attack.']
Original sentence: ['died', 'yesterday.']
Initial prompt: <|endoftext|> The doctor was very famous.He died
Current sentence: ['died', 'yesterday.']
Token IDs: [[3724], [7415], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died
Normalized probabili

In [11]:
import torch
from collections import OrderedDict

def calculate_probabilities_with_prompt_v2(split_lists, index, tokenizer, model, device, symbol_choosen):
    # Tomar la lista de palabras en la posición 'index' de split_lists
    sentence = split_lists[index]
    print(f"Original sentence: {sentence}")

    # Definir el prompt inicial con symbol_choosen y la primera palabra de la sentence
    prompt = tokenizer.decode(tokenizer.bos_token_id) + " The doctor was very famous." + symbol_choosen + " " + sentence[0]
    print(f"Initial prompt: {prompt}")
    
    # Iterar sobre la lista reduciendo el tamaño en cada paso
    for i in range(len(sentence)):
        # Limpiar word_probs para cada sublista
        word_probs = OrderedDict()

        # Usar solo las palabras desde el índice actual hasta el final
        current_sentence = sentence[i:]

        # Codificar los tokens y organizarlos en la forma [[token_id]]
        symbols_dummy_ids = [tokenizer.encode(sym) for sym in current_sentence]
        symbols_dummy_ids = [[token_id] for sublist in symbols_dummy_ids for token_id in sublist]

        print(f"Current sentence: {current_sentence}")
        print(f"Token IDs: {symbols_dummy_ids}")

        # Actualizar el prompt con la palabra actual de la sentence
        prompt += " " + current_sentence[0]
        print(f"Updated prompt: {prompt}")

        # Codificar el nuevo prompt en tokens
        input_ids = torch.tensor(tokenizer.encode(prompt)).reshape(1, -1).to(device)

        # Recalcular las probabilidades para los tokens actuales
        with torch.no_grad():
            output = model(input_ids)
            logits = output.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)[0]

        # Iterar sobre los token_ids (múltiples sub-tokens para cada símbolo)
        for token_ids in symbols_dummy_ids:
            token_prob = 1.0  # Inicializa la probabilidad para este token compuesto

            for token in token_ids:
                token_prob *= probs[token].item()  # Multiplicar las probabilidades de sub-tokens
                # Recalcular las probabilidades para el siguiente token (si es parte del símbolo)
                input_ids = torch.cat([input_ids, torch.tensor([[token]]).to(device)], dim=1)

                with torch.no_grad():
                    output = model(input_ids)
                    logits = output.logits[:, -1, :]
                    probs = torch.softmax(logits, dim=-1)[0]

            # Almacenar la probabilidad acumulada para este símbolo
            word = tokenizer.decode(token_ids).replace(" ", "")
            word_probs[word] = token_prob

        # Normalizar las probabilidades
        normalized_word_probs_dummy = {}
        total = sum(word_probs.values())
        for word in word_probs:
            normalized_word_probs_dummy[word] = word_probs[word] / total

        print(f"Normalized probabilities for '{' '.join(current_sentence)}': {normalized_word_probs_dummy}")
    
    return normalized_word_probs_dummy


# Ejemplo de uso:
symbol_choosen = np.random.choice(list(normalized_word_probs_he_she.keys()), 
                                  p=list(normalized_word_probs_he_she.values()))







In [12]:
for i in range (4):
    times = []
    for j in range(11):
        start = time.time()
        normalized_word_probs_dummy = calculate_probabilities_with_prompt(split_lists, j, tokenizer, model, device, symbol_choosen)
        elapsed_time = time.time() - start
        times.append(elapsed_time)
        
    print(times)


print(times)

Original sentence: ['died.']
Initial prompt: <|endoftext|> The doctor was very famous.He died.
Current sentence: ['died.']
Token IDs: [[3724], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died. died.
Normalized probabilities for 'died.': {'died': 0.9915526484211178, '.': 0.008447351578882196}
Original sentence: ['died', 'yesterday.']
Initial prompt: <|endoftext|> The doctor was very famous.He died
Current sentence: ['died', 'yesterday.']
Token IDs: [[3724], [7415], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died
Normalized probabilities for 'died yesterday.': {'died': 0.0009460281704091932, 'yesterday': 0.3985576131563121, '.': 0.6004963586732787}
Current sentence: ['yesterday.']
Token IDs: [[7415], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday.
Normalized probabilities for 'yesterday.': {'yesterday': 0.04771450905706392, '.': 0.9522854909429361}
Original sentence: ['died', 'yesterday', 'in.']
Init

Normalized probabilities for 'died yesterday in a very sad way due.': {'died': 2.521122242014301e-05, 'yesterday': 0.010621379940703989, 'in': 0.8054416175194553, 'a': 0.15556053323467678, 'very': 0.009638466706666252, 'sad': 3.075727699859483e-05, 'way': 0.0005326581804516369, 'due': 0.0021464198391591173, '.': 0.016002956079468226}
Current sentence: ['yesterday', 'in', 'a', 'very', 'sad', 'way', 'due.']
Token IDs: [[7415], [287], [257], [845], [6507], [835], [2233], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday
Normalized probabilities for 'yesterday in a very sad way due.': {'yesterday': 0.0007306109967993932, 'in': 0.37514263991274144, 'a': 0.003420794208550386, 'very': 0.0009198518022412188, 'sad': 1.5014614049350267e-05, 'way': 0.0003912767873334704, 'due': 0.0018172138539133767, '.': 0.6175625978243714}
Current sentence: ['in', 'a', 'very', 'sad', 'way', 'due.']
Token IDs: [[287], [257], [845], [6507], [835], [2233], [13]]
Updated prompt: 

Normalized probabilities for 'due to a.': {'due': 0.00010782425980001528, 'to': 0.9990753436372213, 'a': 0.0006856345698779613, '.': 0.0001311975331007358}
Current sentence: ['to', 'a.']
Token IDs: [[284], [257], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due to
Normalized probabilities for 'to a.': {'to': 0.001489012750577929, 'a': 0.9978980626656611, '.': 0.000612924583760976}
Current sentence: ['a.']
Token IDs: [[257], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due to a.
Normalized probabilities for 'a.': {'a': 0.9510133660188812, '.': 0.048986633981118834}
Original sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a', 'heart.']
Initial prompt: <|endoftext|> The doctor was very famous.He died
Current sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a', 'heart.']
Token IDs: [[3724], [7415], [287], [257], [

Normalized probabilities for 'died yesterday in a.': {'died': 2.5526430492935396e-05, 'yesterday': 0.01075417575066956, 'in': 0.8155118035570132, 'a': 0.15750545819969086, '.': 0.016203036062133414}
Current sentence: ['yesterday', 'in', 'a.']
Token IDs: [[7415], [287], [257], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday
Normalized probabilities for 'yesterday in a.': {'yesterday': 0.0007329148097391605, 'in': 0.37632556553509794, 'a': 0.0034315808925675486, '.': 0.6195099387625953}
Current sentence: ['in', 'a.']
Token IDs: [[287], [257], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in
Normalized probabilities for 'in a.': {'in': 0.0010015336294889503, 'a': 0.9985176685792783, '.': 0.0004807977912327528}
Current sentence: ['a.']
Token IDs: [[257], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a.
Normalized probabilities for 'a.': {'a': 0.9418399921573986, '.': 0.0

Normalized probabilities for 'in a very sad way due.': {'in': 0.0009985814927525934, 'a': 0.995574421738113, 'very': 0.0020887785342303842, 'sad': 3.887460551929016e-05, 'way': 0.00021956520051078757, 'due': 0.0006003978449925588, '.': 0.00047938058388148126}
Current sentence: ['a', 'very', 'sad', 'way', 'due.']
Token IDs: [[257], [845], [6507], [835], [2233], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a
Normalized probabilities for 'a very sad way due.': {'a': 0.014795217757477398, 'very': 0.9073241150820428, 'sad': 0.012575905188705952, 'way': 0.06242483215831941, 'due': 0.0019014030114284813, '.': 0.000978526802025937}
Current sentence: ['very', 'sad', 'way', 'due.']
Token IDs: [[845], [6507], [835], [2233], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very
Normalized probabilities for 'very sad way due.': {'very': 0.34732307829138237, 'sad': 0.6429811554699992, 'way': 0.006971397696683215, '

...

Normalized probabilities for 'died yesterday in a very sad way due to.': {'died': 2.511949255219676e-05, 'yesterday': 0.010582734540527388, 'in': 0.8025110554077807, 'a': 0.15499453342183692, 'very': 0.009603397590878339, 'sad': 3.0645368067308215e-05, 'way': 0.000530720128272371, 'due': 0.0021386101897449754, 'to': 0.0036384537971851463, '.': 0.015944730063154697}
Current sentence: ['yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to.']
Token IDs: [[7415], [287], [257], [845], [6507], [835], [2233], [284], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday
Normalized probabilities for 'yesterday in a very sad way due to.': {'yesterday': 0.0007286233710689481, 'in': 0.3741220651240452, 'a': 0.003411487944865254, 'very': 0.0009173493472845589, 'sad': 1.4973766822374589e-05, 'way': 0.0003902123196295393, 'due': 0.0018122701273207268, 'to': 0.0027204979656100404, '.': 0.6158825200333533}
Current sentence: ['in', 'a', 'very', 'sad', 'way', 'due', 'to

Normalized probabilities for 'sad way due to a heart.': {'sad': 0.003272135567139229, 'way': 0.9835660125309224, 'due': 0.00014638744768253616, 'to': 0.001083564912476029, 'a': 0.0005352670048206483, 'heart': 0.008002368881630497, '.': 0.003394263655328707}
Current sentence: ['way', 'due', 'to', 'a', 'heart.']
Token IDs: [[835], [2233], [284], [257], [2612], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way
Normalized probabilities for 'way due to a heart.': {'way': 5.491434716667803e-05, 'due': 0.0015356473253097687, 'to': 0.0016550302164088698, 'a': 0.0006334726468642601, 'heart': 9.703761297524824e-06, '.': 0.9961112317029529}
Current sentence: ['due', 'to', 'a', 'heart.']
Token IDs: [[2233], [284], [257], [2612], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due
Normalized probabilities for 'due to a heart.': {'due': 0.00010782339385802551, 'to': 0.9990673200134784, 'a': 

Normalized probabilities for 'a very sad.': {'a': 0.015812367850413667, 'very': 0.969701352315513, 'sad': 0.013440480711765436, '.': 0.0010457991223078262}
Current sentence: ['very', 'sad.']
Token IDs: [[845], [6507], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very
Normalized probabilities for 'very sad.': {'very': 0.35002124534697254, 'sad': 0.6479761318464294, '.': 0.002002622806598049}
Current sentence: ['sad.']
Token IDs: [[6507], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad.
Normalized probabilities for 'sad.': {'sad': 0.9086128290088457, '.': 0.09138717099115437}
Original sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way.']
Initial prompt: <|endoftext|> The doctor was very famous.He died
Current sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way.']
Token IDs: [[3724], [7415], [287], [257], [845], [6507], [835], [13]]
Updated prompt: <|endoftext|> The doc

Normalized probabilities for 'way due to.': {'way': 5.494968951060707e-05, 'due': 0.001536635653109789, 'to': 0.0016560953778855915, '.': 0.996752319279494}
Current sentence: ['due', 'to.']
Token IDs: [[2233], [284], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due
Normalized probabilities for 'due to.': {'due': 0.0001078982385624026, 'to': 0.9997608142130551, '.': 0.00013128754838250136}
Current sentence: ['to.']
Token IDs: [[284], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due to.
Normalized probabilities for 'to.': {'to': 0.6624672859238168, '.': 0.3375327140761832}
Original sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a.']
Initial prompt: <|endoftext|> The doctor was very famous.He died
Current sentence: ['died', 'yesterday', 'in', 'a', 'very', 'sad', 'way', 'due', 'to', 'a.']
Token IDs: [[3724], [7415], [287], [257], [845], [6507

Normalized probabilities for 'sad way due to a heart.': {'sad': 0.003272135567139229, 'way': 0.9835660125309224, 'due': 0.00014638744768253616, 'to': 0.001083564912476029, 'a': 0.0005352670048206483, 'heart': 0.008002368881630497, '.': 0.003394263655328707}
Current sentence: ['way', 'due', 'to', 'a', 'heart.']
Token IDs: [[835], [2233], [284], [257], [2612], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way
Normalized probabilities for 'way due to a heart.': {'way': 5.491434716667803e-05, 'due': 0.0015356473253097687, 'to': 0.0016550302164088698, 'a': 0.0006334726468642601, 'heart': 9.703761297524824e-06, '.': 0.9961112317029529}
Current sentence: ['due', 'to', 'a', 'heart.']
Token IDs: [[2233], [284], [257], [2612], [13]]
Updated prompt: <|endoftext|> The doctor was very famous.He died died yesterday in a very sad way due
Normalized probabilities for 'due to a heart.': {'due': 0.00010782339385802551, 'to': 0.9990673200134784, 'a': 